In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torch.autograd import Variable
from torchvision import datasets,models, transforms
import os
import numpy as np

In [ ]:
# Data augmentation and normalization for training
# Just normalize for validation
# class torchvision.transforms.RandomResizedCrop(size, scale = (0.08,1.0), ratio = (0.75,1.333333333333), interpolation = 2)
# transforms.RandomResizedCrop(224) --> A crop of random size (default of 0.08 to 1.0) of the original size and a 
# random aspect ratio (default of 3/4 to 4/3) of the original aspect ratio is made
# This crop is finally resized to given size (224 in this case)
# transforms.CenterCrop(224) --> Crops the image at the center. 224 is the desired output size of the crop

# class torchvision.transforms.Normalize(mean,std)
# Normalize a tensor image with mean and standard deviation. Given mean: (M1,..., Mn) and std: (S1,...,Sn) for n channels
# this transform will normalize each channel of the input torch.Tensor i.e.
# output[channel] = (input[channel] - mean[channel]) / std[channel]

# Parameters mean (sequence) --->  Sequence of means for each channel
#            std (sequence)  --->  Sequence of standard deviations for each channel

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    ]),
    'val':  transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    ])
}

Loading the Dataset with labels based on folder names (super important) (also remember to download the dataset from udemy, lesson 150)

In [ ]:
data_dir = 'hymenoptera_data'
# Create a dictionary that contains the information of the images in both the training and validation set
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train','val']}
# Create a dictionary that contains the data loader
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                              batch_size = 4,
                                              shuffle = True) for x in ['train','val']}

# Create a dictionary that contains the size of each dataset (training and validation)
dataset_sizes = {x: len(image_datasets[x]) for x in ['train','val']}
# Get the class names
class_names = image_datasets['train'].classes
# Print out the results 
print('Class names {}'.format(class_names))
print('There are {} batches in the training set'.format(len(dataloaders['train'])))
print('There are {} batches in the test set'.format(len(dataloaders['val'])))
print('There are {} training images'.format(dataset_sizes['train']))
print('There are {} testing images'.format(dataset_sizes['val']))


Loading and Modifying a Pre-Trained Network

In [ ]:
# Load the ResNet
model_conv = torchvision.models.resnet18(pretrained = True) # Resnet with 18 layers

In [ ]:
# Freeze all layers in the network
for param in model_conv.parameters():
    param.requires_grad = False # This prevents backpropagation in all layers

In [ ]:
# Get the number of inputs of the last layer (or number of neurons in the layer preceding last layer)
num_ftrs = model_conv.fc.in_features # number of inputs features that enter the fully connected part of the resnet
# Reconstruct the last layer (output layer) to have only two classes
model_conv.fc = nn.Linear(num_ftrs, 2)

In [ ]:
# Move the network to the GPU if available
if torch.cuda.is_available():
    model_conv = model_conv.cuda()